In [ ]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

ITEM TO ITEM COLLABORATIVE FILTERING APPROACH 

In [ ]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990209 entries, 0 to 990208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userID   990209 non-null  int64
 1   movieID  990209 non-null  int64
 2   rating   990209 non-null  int64
dtypes: int64(3)
memory usage: 22.7 MB


In [ ]:
Uitem = data.pivot("movieID", "userID", "rating")
Uitem.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
U_scaled = Uitem - Uitem.mean(axis=1).values.reshape(-1, 1)
U_scaled.fillna(0, inplace=True)
U_scaled.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.0,0.0,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.0,0.0,0.854713,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.854713,0.0,0.854713,0.0,0.0,0.854713,0.0,0.0,0.0,-2.145287,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.854713,0.0,0.854713,0.0,0.854713,0.0,0.0,0.0,0.0,0.0,-1.145287,0.0,-1.145287,0.0,0.0,-0.145287,0.0,-1.145287,0.0,0.854713,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.854713,-0.145287,0.0,-0.145287,0.0,0.0,0.0,0.0,0.854713
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.8,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,-1.200000,0.0,0.0,0.0,0.0,-0.2,...,1.8,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-2.2,0.000000,0.0,0.0,-1.200000,0.0,0.000000,0.0,0.000000,-1.200000,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,1.800000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,-2.021142,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021142,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-1.021142,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.021142,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,-0.733728,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266272,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.266272,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.266272,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,-2.017007,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.982993,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [ ]:
train_size = 500
train = U_scaled[:train_size]
test = U_scaled[train_size:]

1- First Structure - With Default Metric

In [ ]:
nn = nei.NearestNeighbors(n_neighbors=train_size)
nn.fit(train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=500, p=2,
                 radius=1.0)

In [ ]:
myPreds = []
realLabels = []
k = 10
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:  
    #userId = 501
    #movieId = 2
    if not np.isnan(Uitem.loc[movieId, userId]): 
      test_user = test.loc[movieId:movieId, :]
      indices = nn.kneighbors(test_user, return_distance=False)[0]

      ratings = []
      i = 0
      while len(ratings) < k and i < len(indices):
        if not np.isnan(Uitem.iloc[indices[i], userId]):
          ratings.append(Uitem.iloc[indices[i], userId])
        i += 1

      if len(ratings) > 0:
        pred = sum(ratings)/len(ratings)
        myPreds.append(pred)

        real_label = Uitem.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.4341595840392336

In [ ]:
myPreds = []
realLabels = []
k = 20
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:  
    #userId = 501
    #movieId = 2
    if not np.isnan(Uitem.loc[movieId, userId]): 
      test_user = test.loc[movieId:movieId, :]
      indices = nn.kneighbors(test_user, return_distance=False)[0]

      ratings = []
      i = 0
      while len(ratings) < k and i < len(indices):
        if not np.isnan(Uitem.iloc[indices[i], userId]):
          ratings.append(Uitem.iloc[indices[i], userId])
        i += 1

      if len(ratings) > 0:
        pred = sum(ratings)/len(ratings)
        myPreds.append(pred)

        real_label = Uitem.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.5723489853140595

2- Second Structure

In [ ]:
k = 5
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.22489386770476327

In [ ]:
k = 10
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.21461326069247022

In [ ]:
k = 15
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.21286247836207467

In [ ]:
k = 20
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.21228926504805687

In [ ]:
k = 25
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.2114687770501381

In [ ]:
k = 50
myPreds = []
realLabels = []
for movieId in test.index:
  print(movieId)
  if movieId > 520:
    break
  for userId in test.columns:
    if not np.isnan(test.loc[movieId, userId]):
      #userId = 501
      #movieId = 2

      filtered_train = train.dropna(subset=[userId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        movieIds = filtered_train.iloc[indices, :].index
        pred = train.loc[movieIds, userId].values.mean()
        myPreds.append(pred)

        real_label = test.loc[movieId, userId]
        realLabels.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.21153753770138012

Prediction with Item to Item Collaborative Filtering 

In [ ]:
test = pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [ ]:
x = pd.DataFrame(test.IDs.str.split('_',1).tolist(),
                         columns = ['userId','movieId'])
print(x)

     userId movieId
0       629    2683
1       601     904
2      5673    3717
3      5616    1721
4      1099    3697
...     ...     ...
9994   4198    3671
9995    566    3148
9996   5531    2105
9997   4650     377
9998   4919    2094

[9999 rows x 2 columns]


In [ ]:
df_test = pd.DataFrame(x)
df_test

,userId,movieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697
...,...,...
9994,4198,3671
9995,566,3148
9996,5531,2105
9997,4650,377


In [ ]:
df_test['userId'] = df_test['userId'].astype(str).astype(int)
df_test['movieId'] = df_test['movieId'].astype(str).astype(int)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   userId   9999 non-null   int64
 1   movieId  9999 non-null   int64
dtypes: int64(2)
memory usage: 156.4 KB


In [ ]:
myPreds = []
k = 10
for i in range(len(df_test.index)):
  userId = df_test.loc[i].at["userId"]
  movieId = df_test.loc[i].at["movieId"]
  filtered_train = Uitem.dropna(subset=[userId])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)

  if k < len(filtered_train):
   nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
   nn.fit(filtered_train)

   test_row = Uitem.loc[movieId:movieId, :]
   test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
   test_row.fillna(0, inplace=True)

   indices = nn.kneighbors(test_row, return_distance=False)[0]

   movieIds = filtered_train.iloc[indices, :].index
   pred = Uitem.loc[movieIds, userId].values.mean()
   myPreds.append(pred)

In [ ]:
myPreds = []
k = 15
for i in range(len(df_test.index)):
  userId = df_test.loc[i].at["userId"]
  movieId = df_test.loc[i].at["movieId"]
  filtered_train = Uitem.dropna(subset=[userId])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)

  if k < len(filtered_train):
   nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
   nn.fit(filtered_train)

   test_row = Uitem.loc[movieId:movieId, :]
   test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
   test_row.fillna(0, inplace=True)

   indices = nn.kneighbors(test_row, return_distance=False)[0]

   movieIds = filtered_train.iloc[indices, :].index
   pred = Uitem.loc[movieIds, userId].values.mean()
   myPreds.append(pred)

In [ ]:
len(myPreds)

9999

In [ ]:
test3 = test.copy()
test3["rating"]= myPreds

In [ ]:
test3.head()

,IDs,rating
0,629_2683,3.800000
1,601_904,4.533333
2,5673_3717,3.400000
3,5616_1721,3.533333
4,1099_3697,2.800000


In [ ]:
from google.colab import files
test3.to_csv('submission15.csv',index=False) 
files.download('submission15.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>